In [ ]:
from datascience import *
%matplotlib inline
path_data = '../../../assets/data/'
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=np.VisibleDeprecationWarning)

# Lecture 21

In [ ]:
botox = Table.read_table('bta.csv')
botox.show()

In [ ]:
botox.group('Group',np.average)

In [ ]:
def difference_of_means(table, label, group_label):
    """Takes: name of table, column label of numerical variable,
    column label of group-label variable
    Returns: Difference of means of the two groups"""
    
    #table with the two relevant columns
    reduced = table.select(label, group_label)  
    
    # table containing group means
    means_table = reduced.group(group_label, np.average)
    # array of group means
    means = means_table.column(1)
    
    return means.item(1) - means.item(0)

In [ ]:
def one_simulated_difference(table, label, group_label):
    """Takes: name of table, column label of numerical variable,
    column label of group-label variable
    Returns: Difference of means of the two groups after shuffling labels"""
    
    # array of shuffled labels
    shuffled_labels = table.sample(with_replacement = False).column(group_label)
    
    # table of numerical variable and shuffled labels
    shuffled_table = table.select(label).with_column('Shuffled Label', shuffled_labels)
    
    return difference_of_means(shuffled_table, label, 'Shuffled Label')   

# Testing the Hypothesis

In [ ]:
observed_diff = difference_of_means(botox, 'Result', 'Group')
observed_diff

In [ ]:
one_simulated_difference(botox, 'Result', 'Group')

In [ ]:
simulated_diffs = make_array()

for i in np.arange(10000):
    sim_diff = one_simulated_difference(botox, 'Result', 'Group')
    simulated_diffs = np.append(simulated_diffs, sim_diff)

In [ ]:
col_name = 'Distances between groups'
Table().with_column(col_name, simulated_diffs).hist(col_name)
plots.scatter(observed_diff,-0.01, c='red')

In [ ]:
# p-value
sum(simulated_diffs >= observed_diff)/len(simulated_diffs)

# Activity:
Simulate the scenario described in the slides. 
There are 2000 people in a room. Each person tests

**Null**: The coin is fair

**Alternative**: The coin is unfair based on 100 tosses of a coin, 

**Test statistic** | number of heads - 50 |, and the 5% cutoff for the P-value.

Suppose all coins are fair. About how many people will conclude that their coins are unfair?

In [ ]:
num_people = ...
num_tosses = ...
fair_coin = ..

In [ ]:
def get_one_statistic():
    num_heads_in_100_tosses = ...
    return ...

In [ ]:
# Generate statistics for the 2000 people in the room
peoples_statistics = make_array()

for ... in ...:
    peoples_statistics = ...

In [ ]:
# Now perform the hypothesis test (simulate under the null hypothesis)
simulated_statistics = make_array()

for ... in ...:
    simulated_statistics = ...

In [ ]:
stats_table = Table().with_columns('Simulated distances',simulated_statistics)
stats_table.hist('Simulated distances')

In [ ]:
# 5% cutoff
cutoff_index = int(0.95 * len(simulated_statistics))
cutoff_index
cutoff_value = stats_table.sort('Simulated distances').column(0).item(cutoff_index)
cutoff_value

In [ ]:
np.average(peoples_statistics >= cutoff_value)

## Discussion Question: Super Soda

In [ ]:
def simulate_one_count(sample_size):
    return np.count_nonzero(np.random.choice(['H', 'T'], sample_size) == 'H')
simulate_one_count(200)

In [ ]:
num_simulations = 10000
counts = make_array()
for i in np.arange(num_simulations):
    counts = np.append(counts, simulate_one_count(200))

In [ ]:
trials = Table().with_column('Number of Heads', counts)
trials.hist(right_end=91)
plots.ylim(-0.001, 0.055)
plots.scatter(91, 0, color='red', s=40, zorder=3)
plots.title('Prediction Under the Null');

In [ ]:
np.count_nonzero(counts <= 91)/len(counts)

Conclusion: 